In [ ]:
import pandas as pd

def load_sequences_from_csv(filename, sequence_length=80):
  # Load dataset
  data = pd.read_csv(filename)  # Replace with your dataset path

  # Fill missing values
  # data['province'] = data['province'].fillna('unknown')
  #data['city'] = data['city'].fillna('unknown')

  # Encode categorical variables
  categorical_columns = ['debit_credit']
  for col in categorical_columns:
      data[col] = LabelEncoder().fit_transform(data[col])

  # Normalize numerical variables
  scaler = MinMaxScaler()
  data['amount_cad'] = scaler.fit_transform(data[['amount_cad']])

  # Combine transaction_date and transaction_time into a single datetime feature
  if 'transaction_time' in data.columns:
    data['transaction_datetime'] = pd.to_datetime(data['transaction_date'] + ' ' + data['transaction_time'])
    data = data.sort_values(by=['customer_id', 'transaction_datetime'])
  else:
    data = data.sort_values(by=['customer_id', 'transaction_date'])

  # Drop unused columns
  columns_to_keep = ['customer_id', 'amount_cad', 'debit_credit']
  data = data[columns_to_keep]

  # Group by customer_id and create sequences
  grouped = data.groupby('customer_id')
  sequences = []
  for customer_id, group in grouped:
      group = group.drop(columns=['customer_id']).values
      for i in range(len(group) - sequence_length + 1):
          sequences.append(group[i:i+sequence_length])
  sequences = np.array(sequences)
  return sequences


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

sequence_length = 75
sequences = load_sequences_from_csv("abm.csv", sequence_length)

# Train-test split
X_train, X_test = train_test_split(sequences, test_size=0.2, random_state=42)

# LSTM Autoencoder Model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, activation='relu', input_shape=(sequence_length, X_train.shape[2]), return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=False),
    tf.keras.layers.RepeatVector(sequence_length),
    tf.keras.layers.LSTM(32, activation='relu', return_sequences=True),
    tf.keras.layers.LSTM(64, activation='relu', return_sequences=True),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(X_train.shape[2]))
])
model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the model
history = model.fit(X_train, X_train, epochs=20, batch_size=32, validation_split=0.2, shuffle=True)

# Reconstruction errors
X_test_pred = model.predict(X_test)
test_loss = np.mean(np.power(X_test - X_test_pred, 2), axis=(1, 2))

# Anomaly detection threshold
threshold = np.percentile(test_loss, 95)  # Adjust as needed
anomalies = test_loss > threshold

print(f"Threshold: {threshold}")
print(f"Number of anomalies detected: {np.sum(anomalies)}")

# Suppose your model is defined as 'model'
model.save('model_lstm.h5')



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                       │ (None, 75, 64)              │          17,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_17 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ repeat_vector_4 (RepeatVector)       │ (None, 75, 32)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_18 (LSTM)                       │ (None, 75, 32)              │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 75, 64)              │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_4 (TimeDistributed) │ (None, 75, 2)               │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 62,850 (245.51 KB)

 Trainable params: 62,850 (245.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 61s 407ms/step - loss: 0.4087 - val_loss: 0.3585
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - loss: 0.3245 - val_loss: 0.1365
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 157ms/step - loss: 0.1505 - val_loss: 0.1216
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.1323 - val_loss: 0.1371
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step - loss: 0.1300 - val_loss: 0.0915
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0865 - val_loss: 0.0965
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0852 - val_loss: 0.0824
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 159ms/step - loss: 0.0853 - val_loss: 0.0814
Epoch 9/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 152ms/step - loss: 0.0798 - val_loss: 0.0811
Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 164ms/step - loss: 0.0771 - val_loss: 0.0789
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 153ms/step - loss: 0.0770 - val_loss: 0.0779
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 165ms/step - loss: 0.0752 - val_l

Threshold: 0.12796705262631766
Number of anomalies detected: 4


In [15]:
sequences = load_sequences_from_csv("cheque.csv", sequence_length)
X_train, X_test = train_test_split(sequences, test_size=0.2, random_state=42)

# Train the model more
history = model.fit(X_train, X_train, epochs=20, batch_size=32, validation_split=0.2, shuffle=True)

# Reconstruction errors
X_test_pred = model.predict(X_test)
test_loss = np.mean(np.power(X_test - X_test_pred, 2), axis=(1, 2))

# Anomaly detection threshold
threshold = np.percentile(test_loss, 95)  # Adjust as needed
anomalies = test_loss > threshold

print(f"Threshold: {threshold}")
print(f"Number of anomalies detected: {np.sum(anomalies)}")


Epoch 1/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 267s 154ms/step - loss: 18161286.0000 - val_loss: 0.1121
Epoch 2/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 256s 147ms/step - loss: 0.1121 - val_loss: 0.1120
Epoch 3/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 254s 146ms/step - loss: 0.1120 - val_loss: 0.1119
Epoch 4/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 260s 150ms/step - loss: 0.1115 - val_loss: 0.1105
Epoch 5/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 255s 147ms/step - loss: 0.1103 - val_loss: 0.1097
Epoch 6/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 252s 145ms/step - loss: 0.1099 - val_loss: 0.1096
Epoch 7/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 251s 145ms/step - loss: 0.1096 - val_loss: 0.1093
Epoch 8/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 257s 148ms/step - loss: 0.1091 - val_loss: 0.1077
Epoch 9/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 255s 147ms/step - loss: 0.1074 - val_loss: 0.1065
Epoch 10/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 253s 146ms/step - loss: 0.1065 - val_loss: 0.1056
Epoch 11/20
1737/1737 ━━━━━━━━━━━━━━━━━━━━ 251s 144ms/step - loss: 0.105